## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BSBSSSSS'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
26
+-------------------+-------------------+------------------+---------+--------+--------+---------+-----------+--------------------+--------------------+--------------------+------------------+-----------+---------+------+--------------------+------+
|            coeff_1|            coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|               res_1|               res_2|               res_3|          res_mean|    station|total_var|undefs|              vector|  year|
+-------------------+-------------------+------------------+---------+--------+--------+---------+-----------+--------------------+--------------------+--------------------+------------------+-----------+---------+------+--------------------+------+
|-50.526022173461065|0.09513012649440823|0.0749061796968224|    132.6|BSBSSSSS| 31.7003| -83.6231|       SNWD|1.548405388636713...|1.512961718978203...|1.490986266402616...|0.98

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USC00090406,1958.0,0.095130
1,USW00093861,1958.0,0.095130
2,USC00096087,1958.0,0.095130
3,USC00097276,1958.0,0.053067
4,USW00093805,1958.0,0.053067


In [8]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(20)

station,USC00010008,USC00080804,USC00081544,USC00085879,USC00087429,USC00087950,USC00090406,USC00090979,USC00091463,USC00092153,USC00092736,USC00095585,USC00096087,USC00097276,USC00098703,USW00012832,USW00013815,USW00093805,USW00093861
year,,,,,,,,,,,,,,,,,,,
1899.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.288489,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1934.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-14.516054,NaN,NaN,NaN,NaN
1955.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.203882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1958.0,0.138876,0.097794,NaN,NaN,NaN,0.053067,0.09513,NaN,NaN,NaN,NaN,NaN,0.095130,0.053067,NaN,0.138876,0.138876,0.053067,0.09513
1968.0,0.204098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973.0,NaN,NaN,NaN,NaN,-48.650598,NaN,NaN,-87.284621,NaN,-97.482137,NaN,NaN,NaN,NaN,NaN,NaN,-55.855007,NaN,NaN
1977.0,NaN,NaN,0.210646,NaN,NaN,NaN,NaN,0.229876,NaN,NaN,0.204928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989.0,NaN,NaN,NaN,0.515797,NaN,NaN,NaN,NaN,1.882013,0.864048,NaN,NaN,0.515797,NaN,NaN,NaN,NaN,NaN,NaN
1993.0,0.241168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [9]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  29.628493192
RMS removing mean-by-station=  21.0384988239
RMS removing mean-by-year   =  8.06147987737


In [10]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 29.628493192
0 after removing mean by year    = 8.06147987737
0 after removing mean by stations= 4.79974625751
1 after removing mean by year    = 4.25278164248
1 after removing mean by stations= 3.98440220781
2 after removing mean by year    = 3.83203849166
2 after removing mean by stations= 3.73580683949
3 after removing mean by year    = 3.67004117905
3 after removing mean by stations= 3.62214940071
4 after removing mean by year    = 3.58540424732
4 after removing mean by stations= 3.55601835965
